# Notebook: Análise Exploratória e Pré-processamento dos Dados

## Introdução

Este notebook faz parte do projeto do Nanodegree 2025/1 da disciplina de Machine Learning & Inteligência Artificial.
Seu objetivo é explorar e preparar os dados para a tarefa de previsão de evasão de estudantes em um curso online síncrono promovido pela PensComp.

---

In [ ]:
pip install unidecode

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unidecode

### Carregamento dos Dados

O dataset original é carregado a partir de um arquivo CSV contendo informações dos usuários. Este arquivo servirá como base para a análise exploratória e limpeza inicial dos dados.

In [2]:
df = pd.read_csv('dados_projeto_evasao_treino - Copia.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'dados_projeto_evasao_treino - Copia.csv'

### Entendimento Inicial

In [ ]:
# Estrutura do DF
df.shape
df.info()

In [ ]:
# Amostra dos dados
df.head()

In [ ]:
# Classificação das colunas por tipo de prefixo
categoricas = [col for col in df.columns if col.startswith("ds_")]
temporais = [col for col in df.columns if col.startswith("ts_")]
numericas_nr = [col for col in df.columns if col.startswith("nr_")]
numericas_vl = [col for col in df.columns if col.startswith("vl_")]

print("\nColunas categóricas:", categoricas)
print("Colunas temporais:", temporais)
print("Colunas numéricas (contagem - nr_):", numericas_nr)
print("Colunas numéricas (valores/escalares - vl_):", numericas_vl)

### Verificação de itens ausentes

In [ ]:
# Verificando colunas que possuam ao menos um item nulo
missing = (df.isnull().mean() * 100).sort_values(ascending=True)

print("\nProporção de valores ausentes por coluna (%):")
print(missing[missing > 0].round(2))

Primeiro vamos remover a coluna que representaria o ID para não influenciar na predição e após isso remover todas as linhas onde na coluna ts_primeiro_acesso esteja zerado "0", pois isso simboliza que o aluno nunca acessou o portal

In [ ]:
df = df.drop("Unnamed: 0", axis=1)


In [ ]:
df = df[df['ts_primeiro_acesso'] != 0]

In [ ]:
df.head()

Agora fizemos a conversao dos dados TS para um datetime

In [ ]:
df['ts_primeiro_acesso'] = pd.to_datetime(df['ts_primeiro_acesso'], unit='s')
df['ts_ultimo_acesso'] = pd.to_datetime(df['ts_ultimo_acesso'], unit='s')

In [ ]:
df.head()

limpeza e formatação coluna cidade usuario

In [ ]:
print(df['ds_cidade_usuario'].unique())

In [ ]:
df['ds_cidade_usuario'] = (
    df['ds_cidade_usuario']
    .astype(str)
    .str.strip()
    .str.upper()
    .apply(unidecode.unidecode)
)

In [ ]:
df['ds_cidade_usuario'].replace("NAN", "CIDADE NAO INFORMADA", inplace=True)

In [ ]:
print(df['ds_cidade_usuario'].unique())

### Tratamento de Valores Ausentes

Durante a análise, identificamos 5 colunas com altos índices de valores nulos:

- `vl_media_submissoes_codigo` (~99%)
- `vl_submissoes_por_dias_ativos` (~98%)
- `vl_desempenho_questionario` (~93%)
- `vl_engajamento_usuario_por_intervalo` (~92.75%)
- `vl_engajamento_usuario_intradia` (~92.75%)

#### Colunas removidas
As duas primeiras foram removidas por conterem dados ausentes em quase todos os registros, o que inviabiliza sua utilização estatística ou em modelos preditivos.

#### Colunas preenchidas com zero
As demais colunas foram mantidas e os valores nulos preenchidos com **zero**, assumindo que a ausência dos dados representa **falta de atividade do usuário** (ex: nenhum questionário feito, nenhum engajamento detectado). Isso mantém a coerência da análise com o objetivo de detectar evasão por inatividade.


In [ ]:
# Remover colunas com mais de 70% de valores ausentes
df.drop(columns=["vl_media_submissoes_codigo", "vl_submissoes_por_dias_ativos"], inplace=True)
df.drop(columns=["vl_media_questoes_por_dia", "vl_engajamento_notas"], inplace=True)

In [ ]:
# Preencher a coluna de desempenho com zero
df["vl_desempenho_questionario"] = df["vl_desempenho_questionario"].fillna(0)
df["vl_engajamento_usuario_por_intervalo"] = df["vl_engajamento_usuario_por_intervalo"].fillna(0)
df["vl_engajamento_usuario_intradia"] = df["vl_engajamento_usuario_intradia"].fillna(0)
df["vl_desempenho_usuario"] = df["vl_desempenho_usuario"].fillna(0)
df["vl_media_notas"] = df["vl_media_notas"].fillna(0)

### Por que utilizamos a mediana para preencher valores ausentes?

Ao lidar com variáveis contínuas como tempo médio de questionário (`vl_medio_tempo_questionario`) e tempo médio em questionários avaliados (`vl_medio_tempo_questionario_avaliado`), optamos por preencher os valores ausentes com a **mediana**.

A **mediana** é o valor central de uma distribuição ordenada — ou seja, separa os 50% menores dos 50% maiores valores. Diferente da **média**, a mediana **não é afetada por outliers ou valores extremos**, o que a torna mais robusta para representar o "comportamento típico" dos dados, especialmente em distribuições assimétricas ou com grande variação.

#### Vantagens de usar a mediana:
- Evita distorção causada por tempos muito longos ou muito curtos.
- Mantém a coerência estatística da variável.
- Preserva o padrão geral da distribuição sem inflar ou achatar artificialmente os dados.

Dessa forma, preenchemos os dados faltantes sem comprometer a qualidade da análise ou da modelagem futura.


In [ ]:
#Preencher com mediana (tempo médio tem sentido contínuo e comparável):
df["vl_medio_tempo_questionario"] = df["vl_medio_tempo_questionario"].fillna(df["vl_medio_tempo_questionario"].median())
df["vl_medio_tempo_questionario_avaliado"] = df["vl_medio_tempo_questionario_avaliado"].fillna(df["vl_medio_tempo_questionario_avaliado"].median())

In [ ]:
df.head()

In [ ]:
df.to_csv("dados_convertidos.csv", index=False)


### Análise de Engajamento por Cidade

Comparamos a média de interações dos alunos por cidade, permitindo identificar regiões com maior ou menor participação.

In [ ]:
# Média de engajamento por cidade
city_stats = df.groupby('ds_cidade_usuario')[['nr_interacoes_usuario',
                                              'vl_desempenho_usuario',
                                              'nr_questionarios_finalizados']].mean().reset_index()

# Gráfico de barras
plt.figure(figsize=(12, 6))
sns.barplot(data=city_stats, x='ds_cidade_usuario', y='nr_interacoes_usuario')
plt.xticks(rotation=45)
plt.title('Média de Interações por Cidade')
plt.xlabel('Cidade')
plt.ylabel('Média de Interações')
plt.show()

### Distribuição do Desempenho Geral dos Alunos

O gráfico de desempenho mostra como os alunos estão performando academicamente, com base na métrica `vl_desempenho_usuario`, que varia de 0 a 1. Indicando uma concentração de alunos com ótimo desempenho e muitos com nenhum

In [ ]:
df['vl_desempenho_usuario'] = pd.to_numeric(df['vl_desempenho_usuario'], errors='coerce')
media_geral = df['vl_desempenho_usuario'].mean()

plt.figure(figsize=(10, 6))
sns.histplot(df['vl_desempenho_usuario'], bins=30, kde=True, color='skyblue')
plt.axvline(media_geral, color='red', linestyle='--', linewidth=2, label=f'Média: {media_geral:.2f}')
plt.title('Distribuição do Desempenho dos Alunos')
plt.xlabel('Desempenho')
plt.ylabel('Frequência')
plt.legend()
plt.tight_layout()
plt.show()


### Distribuição dos Dias Desde o Último Acesso

Esse gráfico analisa o comportamento de uso da plataforma, mostrando há quantos dias cada usuário realizou seu último acesso.

In [ ]:

# Selecionar  a coluna de dias desde o último acesso
dias = df['nr_dias_desde_ultimo_acesso']

# Calcular a média
media_dias = dias.mean()

# Plotar histograma com KDE (curva de densidade)
plt.figure(figsize=(10, 6))
sns.histplot(dias, bins=30, kde=True, color='skyblue')

# Adicionar linha da média
plt.axvline(media_dias, color='red', linestyle='--', linewidth=2, label=f'Média: {media_dias:.2f} dias')

# Títulos e eixos
plt.title("Distribuição dos Dias Desde o Último Acesso dos Usuários")
plt.xlabel("Dias Desde Último Acesso")
plt.ylabel("Frequência")
plt.legend()
plt.tight_layout()
plt.show()


Com base nessas análises sobre engajamento e desempenho dos alunos, criaremos uma variável chamada `evadiu`.  
Essa variável simula o comportamento de evasão, permitindo que possamos identificar alunos com maior risco de abandono do curso.

#### Criação da variável `evadiu`

Criamos uma nova coluna `evadiu` para simular o comportamento de evasão, utilizando a seguinte lógica:

> Se o aluno **nunca obteve desempenho** (`vl_desempenho_usuario = 0`) e está **há mais de 60 dias sem acessar a plataforma**, consideramos que ele evadiu.

Essa coluna é binária:
- `1` → aluno evadiu
- `0` → aluno ativo ou ainda engajado


In [ ]:
# Criar coluna 'evadiu': alunos com desempenho 0 e sem acessar há mais de 60 dias
df["vl_desempenho_usuario"] = pd.to_numeric(df["vl_desempenho_usuario"], errors="coerce")
df["evadiu"] = ((df["vl_desempenho_usuario"] == 0) & (df["nr_dias_desde_ultimo_acesso"] > 60)).astype(int)


#### Comparação entre evasores e não evasores

Com a variável `evadiu` criada, comparamos os dois grupos em relação a:
- Número de interações
- Questionários finalizados
- Submissões de código
- Desempenho geral

Isso nos ajuda a entender quais comportamentos estão mais associados à evasão.

In [ ]:
# Comparar médias de variáveis entre evasores e não evasores
media_por_grupo = df.groupby("evadiu")[
    ["nr_interacoes_usuario", "nr_questionarios_finalizados", "vl_desempenho_usuario", "nr_submissoes_codigo"]
].mean().round(2)

media_por_grupo


#### Classificação de Perfis de Risco

A partir das variáveis `vl_desempenho_usuario` e `nr_dias_desde_ultimo_acesso`, classificamos cada aluno em um dos seguintes perfis:

| Perfil            | Critério                                              |
|-------------------|-------------------------------------------------------|
| Alto Risco        | Desempenho = 0 e sem acesso há mais de 60 dias        |
| Reengajamento     | Bom desempenho, mas inativo há mais de 60 dias        |
| Apoio Pedagógico  | Acessa, mas desempenho muito baixo                    |
| Estável           | Acessa com frequência e tem bom desempenho            |


In [ ]:
# Classificar alunos em perfis com base em desempenho e atividade
def perfil(row):
    if row["vl_desempenho_usuario"] == 0 and row["nr_dias_desde_ultimo_acesso"] > 60:
        return "Alto Risco"
    elif row["vl_desempenho_usuario"] > 0.7 and row["nr_dias_desde_ultimo_acesso"] > 60:
        return "Reengajamento"
    elif row["vl_desempenho_usuario"] < 0.3:
        return "Apoio Pedagógico"
    else:
        return "Estável"

df["perfil"] = df.apply(perfil, axis=1)


#### Contagem de Perfis

Finalizamos com a contagem de alunos em cada perfil. Isso nos ajuda a:

- Visualizar a distribuição de risco na base de dados
- Compreender o tamanho de cada grupo e onde priorizar intervenções

Essa segmentação é essencial tanto para análise quanto para futuras estratégias de reengajamento ou previsão automatizada da evasão.

In [ ]:
# Contar a quantidade de alunos em cada perfil
df["perfil"].value_counts().reset_index().rename(columns={"index": "perfil", "perfil": "quantidade"})


### Pré-Processamento dos Dados para Modelagem

**Seleção de Atributos**: removemos variáveis que não serão usadas como entrada no modelo, como:
   - `evadiu` (alvo da predição)
   - `perfil` (já é um agrupamento explicativo derivado das features)
   - `ds_cidade_usuario` (categórica não transformada)

In [ ]:
y = df["evadiu"]

In [ ]:
X = df.drop(columns=["evadiu", "perfil", "ds_cidade_usuario"], errors="ignore")

**Exclusão de colunas do tipo `datetime64`:**
   - Modelos de ML não entendem datas diretamente.
   - Em vez disso, já transformamos essas colunas em atributos úteis como `nr_dias_desde_ultimo_acesso`.

In [ ]:
X = X.select_dtypes(exclude=["datetime64[ns]"])

**Escalonamento de Variáveis Numéricas**: usamos `MinMaxScaler` para normalizar todas as variáveis entre 0 e 1. Isso é especialmente importante para algoritmos como redes neurais, que são sensíveis à escala dos dados.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

**Separação dos Conjuntos de Treino e Teste**: dividimos os dados em 80% para treino e 20% para teste, garantindo que a proporção de evasores e não evasores seja mantida (com `stratify`).

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

**Verificação de Balanceamento**: imprimimos a distribuição das classes (evadiu = 0 ou 1) no conjunto de treino, o que ajuda a identificar se técnicas adicionais de balanceamento serão necessárias no modelo.

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
print("Distribuição de 'evadiu' no treino:", dict(zip(unique, counts)))

# ANALISAR A PARTIR DAQUI

### SEÇÃO: Preparação Final dos Dados
---


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder  # ou OneHotEncoder conforme necessário

# Seleciona as features (X) e o target (y)
# Substitua 'nome_da_coluna_target' pela sua variável de evasão
X = df.drop('nome_da_coluna_target', axis=1)
y = df['nome_da_coluna_target']

# Aplicar encoding nas variáveis categóricas se necessário
# Exemplo com LabelEncoder:
# for col in X.select_dtypes(include='object').columns:
#     X[col] = LabelEncoder().fit_transform(X[col])

# Escalonamento das variáveis numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Separação entre treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


### SEÇÃO: Modelos de Machine Learning Tradicionais


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Exemplo com Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_val)

print("Random Forest:")
print(classification_report(y_val, y_pred_rf))
print(confusion_matrix(y_val, y_pred_rf))

### SEÇÃO: Modelo de Rede Neural com Keras


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Definição da rede neural
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Saída binária

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Treinamento
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Avaliação
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Acurácia na validação: {accuracy:.2f}")

### SEÇÃO: Visualização das Métricas da Rede Neural


In [ ]:
import matplotlib.pyplot as plt

# Plotando a acurácia e perda durante o treinamento
plt.plot(history.history['accuracy'], label='Acurácia - Treino')
plt.plot(history.history['val_accuracy'], label='Acurácia - Validação')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.title('Evolução da Acurácia')
plt.show()

### SEÇÃO: Teste com Conjunto de Teste Final


In [ ]:
# Carregue e processe o dataset de teste da mesma forma que o de treino
# X_test = ...

# Certifique-se de aplicar o mesmo scaler e transformações:
# X_test_scaled = scaler.transform(X_test)

# y_pred_final = model.predict(X_test_scaled)

### SEÇÃO: Explicabilidade com SHAP


In [ ]:
import shap

# Para modelos do scikit-learn:
explainer = shap.Explainer(rf, X_train)
shap_values = explainer(X_val)

shap.summary_plot(shap_values, X_val)

### SEÇÃO: Exportação para Produção


In [ ]:
import joblib

# Salvar o modelo de ML tradicional
joblib.dump(rf, 'modelo_random_forest.pkl')

# Para salvar o modelo Keras:
model.save('modelo_rede_neural.h5')

# Salvar o scaler
joblib.dump(scaler, 'scaler.pkl')